In [9]:
import copy
import open3d as o3d
import numpy as np

### Matching

In [10]:
def draw_registration_result(source, target, transformation= np.identity(4)):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)

    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(source, target, voxel_size = 0.05):
    print(":: Load two point clouds and disturb initial pose.")

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source_down, target_down, source_fpfh, target_fpfh

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

def refine_registration(source, target, result_ransac, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    radius_normal = voxel_size * 2
    source.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    target.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result

pcd_before = o3d.io.read_point_cloud("TestData/test_before/pcd/map_go_all.pcd")
pcd_after = o3d.io.read_point_cloud("TestData/test_after/pcd/map_go_all.pcd")
# draw_registration_result(pcd_before, pcd_after)

voxel_size = 0.5
source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
    pcd_before, pcd_after, voxel_size)

result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
# print(result_ransac)
# draw_registration_result(source_down, target_down, result_ransac.transformation)

result_icp = refine_registration(pcd_before, pcd_after, result_ransac,
                                 voxel_size)
print(result_icp)
draw_registration_result(pcd_before, pcd_after, result_icp.transformation)


:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.500.
:: Estimate normal with search radius 1.000.
:: Compute FPFH feature with search radius 2.500.
:: Downsample with a voxel size 0.500.
:: Estimate normal with search radius 1.000.
:: Compute FPFH feature with search radius 2.500.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.500,
   we use a liberal distance threshold 0.750.
:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.200.
RegistrationResult with fitness=9.983707e-01, inlier_rmse=1.557353e-02, and correspondence_set size of 3097564
Access transformation to get result.


### Change detection

In [11]:
from sklearn.neighbors import NearestNeighbors

In [13]:
pcd_before_t = copy.deepcopy(pcd_before).transform(result_icp.transformation)
draw_registration_result(pcd_before_t, pcd_after)